## Pre-trained Models

Incredibly, many people from across the world have shared the results of their hard work building and training machine learning models, so that you can take their working models straight off the shelf.

> You can download many model architectures and pre-trained weights from online

One of the largest collections of machine learning models can be found on [HuggingFace](https://huggingface.co/models).

Let's download HuggingFace and some related libraries before we start playing with some of those models.

In [1]:
!pip install transformers
!pip install timm
!pip install pillow
!pip install requests

Many of the examples come with implementation code that you can copy directly (always assuming that you understand what it does beforehand, of course).

[Here](https://huggingface.co/SenseTime/deformable-detr) in the documentation, you can find example code that gets the Deformable DETR object detection model from HuggingFace and uses it to make a prediction.

Copying that below allows us to run the model in our own environment:

In [2]:
from transformers import AutoImageProcessor, DeformableDetrForObjectDetection
import torch
from PIL import Image
import requests

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

processor = AutoImageProcessor.from_pretrained("SenseTime/deformable-detr")
model = DeformableDetrForObjectDetection.from_pretrained("SenseTime/deformable-detr")

inputs = processor(images=image, return_tensors="pt")
outputs = model(**inputs)

# convert outputs (bounding boxes and class logits) to COCO API
# let's only keep detections with score > 0.7
target_sizes = torch.tensor([image.size[::-1]])
results = processor.post_process_object_detection(outputs, target_sizes=target_sizes, threshold=0.7)[0]

for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
    box = [round(i, 2) for i in box.tolist()]
    print(
            f"Detected {model.config.id2label[label.item()]} with confidence "
            f"{round(score.item(), 3)} at location {box}"
    )


/Users/ice/opt/miniconda3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
/Users/ice/opt/miniconda3/lib/python3.9/site-packages/transformers/models/deformable_detr/image_processing_deformable_detr.py:792: FutureWarning: The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.
  warnings.warn(
Downloading: 100%|██████████| 4.54k/4.54k [00:00<00:00, 722kB/s]
Downloading: 100%|██████████| 161M/161M [00:05<00:00, 29.9MB/s] 
Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-rsb-weights/resnet50_a1_0-14fe96d1.pth" to /Users/ice/.cache/torch/hub

Detected cat with confidence 0.8 at location [16.5, 52.84, 318.25, 470.78]
Detected cat with confidence 0.789 at location [342.19, 24.3, 640.02, 372.25]


/Users/ice/opt/miniconda3/lib/python3.9/site-packages/transformers/models/deformable_detr/image_processing_deformable_detr.py:1329: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  topk_boxes = topk_indexes // out_logits.shape[2]


## Be careful of filling up your machine!

> Some downloadable models are huge!

Every time you get a model from HuggingFace, you're losing a big chunk of your hard drive.

You'll want to clear out old models from time to time by deleting them from the command line. I found mine in `~/.cache/torch/hub/` on MacOS, and had to delete them when I encountered a `No space left on device` error. For different operating systems, you might find the location described in this [GitHub issue about deleting models](https://github.com/huggingface/transformers/issues/861).

## With great power, comes great responsibility

Despite how easy it is to copy the code provided online, it is important to understand what all of the code provided in the copyable examples does before copying it blindly.

- Make you understand the input format expected by the model
- Make sure you understand the format of the model output

Without these, it will be hard to integrate your model into your larger application, or will make your implementation prone to error.